In [21]:
import torch
from torch import nn

In [1]:
import pandas as pd

splits = {'train': 'plain_text/train-00000-of-00001.parquet', 'test': 'plain_text/test-00000-of-00001.parquet', 'unsupervised': 'plain_text/unsupervised-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/stanfordnlp/imdb/" + splits["train"])

/home/quan-pham/jupyter-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
test_dataset = pd.read_parquet("hf://datasets/stanfordnlp/imdb/" + splits["test"])

In [3]:
from sklearn.model_selection import train_test_split
train_dataset, valid_dataset = train_test_split(df, train_size=20000, test_size=5000)

In [15]:
import re
from collections import Counter, OrderedDict

token_counts = Counter()

def tokenizer(text):
    text = re.sub(r'<[^>]*>', '', text)
    emoticons = re.findall(r'(?::|;|=)(?:-)?(?:\)|\(|d|p)', text.lower())
    text = re.sub(r'[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', '')
    tokenized = text.split()
    return tokenized

In [17]:
for i, (text, label) in train_dataset.iterrows():
    tokens = tokenizer(text)
    token_counts.update(tokens)

print('Vocab-size:', len(token_counts))

Vocab-size: 69255


In [19]:
from torchtext.vocab import vocab
sorted_by_freq_tuples = sorted(token_counts.items(), key=lambda x: x[1], reverse=True)
ordered_dict = OrderedDict(sorted_by_freq_tuples)
vocab = vocab(ordered_dict)
vocab.insert_token("<pad>", 0)
vocab.insert_token("<unk>", 1)
vocab.set_default_index(1)

print([vocab[token] for token in ['this', 'is', 'an', 'example']])

[11, 7, 35, 467]


In [20]:
text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]
label_pipeline = lambda x: float(x)

In [26]:
device = torch.device('cuda')

def collate_batch(batch):
    label_list, text_list, lengths = [], [], []
    for _text, _label in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        lengths.append(processed_text.size(0))
    label_list = torch.tensor(label_list)
    lengths = torch.tensor(lengths)
    padded_text_list = nn.utils.rnn.pad_sequence(text_list, batch_first=True)
    return padded_text_list.to(device), label_list.to(device), lengths.to(device)

In [27]:
from torch.utils.data import Dataset, DataLoader

class TabularDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        return self.dataframe.iloc[idx]['text'], self.dataframe.iloc[idx]['label']

In [29]:
train_set = TabularDataset(train_dataset)
dataloader = DataLoader(train_set, batch_size=4, shuffle=False, collate_fn=collate_batch)
text_batch, label_batch, length_batch = next(iter(dataloader))
print(text_batch)
print(label_batch)
print(length_batch)
print(text_batch.shape)

tensor([[   2, 1570,  406,  ...,    0,    0,    0],
        [  10,  294,    2,  ...,    0,    0,    0],
        [  40,  218,   21,  ...,    0,    0,    0],
        [1961,   11,   17,  ...,    4,   86,   20]], device='cuda:0')
tensor([1., 0., 1., 1.], device='cuda:0')
tensor([517, 167,  71, 522], device='cuda:0')
torch.Size([4, 522])


In [37]:
batch_size = 32
train_set = TabularDataset(train_dataset)
valid_set = TabularDataset(valid_dataset)
test_set = TabularDataset(test_dataset)

train_dl = DataLoader(train_set, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
valid_dl = DataLoader(valid_set, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)
test_dl = DataLoader(test_set, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)

In [33]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.rnn = nn.LSTM(embed_dim, rnn_hidden_size, batch_first=True)
        self.fc1 = nn.Linear(rnn_hidden_size, fc_hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(fc_hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, text, lengths):
        out = self.embedding(text)
        out = nn.utils.rnn.pack_padded_sequence(out, lengths.cpu().numpy(), enforce_sorted=False, batch_first=True)
        out, (hidden, cell) = self.rnn(out)
        out = hidden[-1, :, :]
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

In [40]:
vocab_size = len(vocab)
embed_dim = 20
rnn_hidden_size = 64
fc_hidden_size = 64

model = RNN(vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size).to(device)
model

RNN(
  (embedding): Embedding(69257, 20, padding_idx=0)
  (rnn): LSTM(20, 64, batch_first=True)
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [ ]:
from tqdm import tqdm
def train(dataloader, optimizer, loss_fn):
    model.train()
    total_acc, total_loss = 0, 0
    for text_batch, label_batch, lengths in tqdm(dataloader, "Train   "):
        optimizer.zero_grad()
        pred = model(text_batch, lengths)[:, 0]
        loss = loss_fn(pred, label_batch)
        loss.backward()
        optimizer.step()
        total_acc += ((pred >= 0.5).float() == label_batch).float().sum().item()
        total_loss += loss.item() * label_batch.size(0)
    
    return total_acc / len(dataloader.dataset), total_loss / len(dataloader.dataset)

def evaluate(dataloader, loss_fn):
    model.eval()
    total_acc, total_loss = 0, 0

    with torch.no_grad():
        for text_batch, label_batch, lengths in tqdm(dataloader, "Evaluate"):
            pred = model(text_batch, lengths)[:, 0]
            loss = loss_fn(pred, label_batch)
            total_acc += ((pred >= 0.5).float() == label_batch).float().sum().item()
            total_loss += loss.item() * label_batch.size(0)

    return total_acc / len(dataloader.dataset), total_loss / len(dataloader.dataset)

In [41]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
print(f'Train model on device {torch.cuda.get_device_name(device)}')
for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}:')
    acc_train, loss_train = train(train_dl, optimizer, loss_fn)
    acc_valid, loss_valid = evaluate(valid_dl, loss_fn)
    print(f'accuracy: {acc_train:.4f} val_accuracy: {acc_valid:.4f}')

Train model on device NVIDIA GeForce GTX 1050
Epoch 1:


evaluating...: 100%|██████████| 157/157 [00:04<00:00, 38.35it/s]


accuracy: 0.6132 val_accuracy: 0.6958
Epoch 2:


evaluating...: 100%|██████████| 157/157 [00:04<00:00, 38.78it/s]


accuracy: 0.7028 val_accuracy: 0.7546
Epoch 3:


...

accuracy: 0.9274 val_accuracy: 0.8602
Epoch 9:


evaluating...: 100%|██████████| 157/157 [00:04<00:00, 35.60it/s]


accuracy: 0.9363 val_accuracy: 0.8616
Epoch 10:


evaluating...: 100%|██████████| 157/157 [00:04<00:00, 35.44it/s]

accuracy: 0.9518 val_accuracy: 0.8478


In [42]:
acc_test, _ = evaluate(test_dl, loss_fn)
print(f'test_accuracy: {acc_test:.4f}')

evaluating...: 100%|██████████| 782/782 [00:19<00:00, 40.12it/s]

test_accuracy: 0.8329
